### Fine Tuning Schlager Bot

Using the dataset of colelcted Schlager song lyrics, we will fine tune a German based LLM to generate song lyrics from a provided verse.

In [35]:
import pandas as pd

df_lyrics = pd.read_csv('schlager_songs.csv')

df_lyrics['lyrics'][10]

'{"status"false,"reason""Unexpected error occurred (no quota cost) Please try again later"}'

In [43]:
# Create separate column for 1st verse

verse_list = []

for ind in df_lyrics["lyrics"].index:
    lyrics = df_lyrics["lyrics"].iloc[ind]
    if lyrics == '{"status"false,"reason""Unexpected error occurred (no quota cost) Please try again later"}' :
        verse_list.append(None)
    else:
        try:
            verse = lyrics.split("\n")[0]
            n = 1
            while len(' '.join(verse).split()) <= 15:
                verse = lyrics.split("\n")[0:n]
                n += 1
            verse_list.append(''.join(verse))

        except:
            verse_list.append(None)


df_lyrics["verse_1"] = verse_list

df_lyrics["verse_1"][0]

df_lyrics.to_csv("schlager_songs_v2.csv")


In [4]:
# Prompt Example

### Instruction:

"Benuzte den unten gegebenen Vers um den Text für ein Lied zu schreiben."

### Input:

"Hey Hey Hey (Hey) In einer Kneipe bei Bier und Pfeife Da saßen wir beisamm'n (die ganze Nacht)"

### Response:
f"""Hey Hey Hey (Hey) In einer Kneipe bei Bier und Pfeife Da saßen wir beisamm'n (die ganze Nacht)
Ein kühler Tropfen vom besten Hopfen Uns durch die Kehle rann (die ganze Nacht)
Ja, wenn die Burschen singen und die Klampfen klingen
Und die Mädels fall'n drauf rein (diese dummen Dinger)
Was kann das Leben schöneres geben?
Wir gehen heut nicht heim Sonnenuntergang die ganze Nacht
Und es spielt die Balalaika (-laika) Dir gehört mein Herz die ganze Nacht (hey)
Schwarze Natascha nur du allein
Ja, ja, ja, schwarze Natascha nur du (hey)
Süße Natascha nur du (hey) Dir gehört mein Herz die ganze Nacht (hey)
Schwarze Natascha nur du allein (hey)
(Hey) Es ist sehr spät schon
Der Wirt der schläft schon
Das Bier wird langsam schal
Bevor wir gehen und Abschied nehmen
Da singen wir nochmal Ja, wenn die Burschen singen und die Klampfen klingen
Und die Mädels fall'n drauf rein (diese dummen Dinger)
Was kann das Leben schöneres geben?
Wir gehen heut nicht heim Sonnenuntergang die ganze Nacht Und es spielt die Balalaika (-laika)
Dir gehört mein Herz die ganze Nacht (hey) Schwarze Natascha nur du allein
Ja, ja, ja, schwarze Natascha nur du (hey)
Süße Natascha nur du (hey)
Dir gehört mein Herz die ganze Nacht (hey)
Schwarze Natascha nur du allein
Ja, ja, ja, schwarze Natascha nur du (hey)
Süße Natascha nur du (hey)
Dir gehört mein Herz die ganze Nacht (hey)
Schwarze Natascha nur du allein
Ja, ja, ja, schwarze Natascha nur du (hey)
Süße Natascha nur du (hey)
Dir gehört mein Herz die ganze Nacht (hey)
Schwarze Natascha nur du allein Ja, ja, ja, schwarze Natascha nur du (hey)
Süße Natascha nur du (hey) Dir gehört mein Herz die ganze Nacht (hey)
Schwarze Natascha nur du allein Hey"""

"Hey Hey Hey (Hey) In einer Kneipe bei Bier und Pfeife Da saßen wir beisamm'n (die ganze Nacht)\nEin kühler Tropfen vom besten Hopfen Uns durch die Kehle rann (die ganze Nacht)\nJa, wenn die Burschen singen und die Klampfen klingen\nUnd die Mädels fall'n drauf rein (diese dummen Dinger)\nWas kann das Leben schöneres geben?\nWir gehen heut nicht heim Sonnenuntergang die ganze Nacht\nUnd es spielt die Balalaika (-laika) Dir gehört mein Herz die ganze Nacht (hey)\nSchwarze Natascha nur du allein\nJa, ja, ja, schwarze Natascha nur du (hey)\nSüße Natascha nur du (hey) Dir gehört mein Herz die ganze Nacht (hey)\nSchwarze Natascha nur du allein (hey)\n(Hey) Es ist sehr spät schon\nDer Wirt der schläft schon\nDas Bier wird langsam schal\nBevor wir gehen und Abschied nehmen\nDa singen wir nochmal Ja, wenn die Burschen singen und die Klampfen klingen\nUnd die Mädels fall'n drauf rein (diese dummen Dinger)\nWas kann das Leben schöneres geben?\nWir gehen heut nicht heim Sonnenuntergang die ganze N

In [60]:
def fromat_instruction(sample):
    return f"""### Instruction:
Benuzte den unten gegebenen Vers um den Text für ein Lied zu schreiben.

### Input:

{sample['input']}

### Response:

{sample['output']}

"""

In [ ]:
#Create input and output dataset from dataframe

from datasets import Dataset

df_dataset = df_lyrics[["verse_1", "lyrics"]]

ind_nan = []

for ind in df_dataset.index:
    if df_dataset["verse_1"].iloc[ind] == None:
        ind_nan.append(ind)
    else:
        continue

df_dataset.drop(index=ind_nan, inplace=True)

df_dataset.reset_index(drop=True, inplace=True)

df_dataset.rename(columns={"lyrics" : "output", "verse_1" : "input"}, inplace= True)

dataset = Dataset.from_pandas(df_dataset)

dataset


In [61]:
from random import randrange

print(fromat_instruction(dataset[randrange(len(dataset))]))

### Instruction:
Benuzte den unten gegebenen Vers um den Text für ein Lied zu schreiben.

### Input:

 Du sagst nicht ein Wort

### Response:

 Du sagst nicht ein Wort
 Und deine Hand wischt eine Träne fort
 Und dein leerer Blick
 Sinkt in dein Glas
 Du sitzt hier vor mir
 Und dein Gesicht lässt keinen Zweifel mehr
 Heut sagst du mir
 Dass ich dich verlier'
 Nie war Zeit für dich
 Ich lebte nur in meiner eignen Welt
 Ich weiß, du wirst gehen
 Ich muss dich versteh'n
 Lieb mich ein letztes mal
 Es bleibt mir keine andre Wahl
 Ich weiß, dass ich die Nacht mit dir
 An den Tag verlier'
 Schenk mir die Zeit
 Die uns noch bleibt
 Lieb mich ein letztes Mal
 Lass' mich dich noch einmal spür'n
 Bist du auch morgen nicht mehr hier
 Etwas bleibt von dir
 Wenn ich erwach'
 Alleine erwach'
 Ich hab' so viel Zeit
 Neben dir, an dir vorbei gelebt
 Ich nahm, doch was ich gab
 Zählte nicht viel
 Du hast oft geweint
 Doch ich war blind
 Sah deine Tränen nicht
 Heut ist mir klar
 Wie einsam du warst
 Lie